# Isolation Forest (IF) outlier detector deployment

Wrap a scikit-learn Isolation Forest python model for use as a prediction microservice in seldon-core and deploy on seldon-core running on minikube or a Kubernetes cluster using GCP.

## Dependencies

- [helm](https://github.com/helm/helm)
- [minikube](https://github.com/kubernetes/minikube) 
- [s2i](https://github.com/openshift/source-to-image) >= 1.1.13 

python packages:
- scikit-learn: pip install scikit-learn --> 0.20.1

## Task

The outlier detector needs to detect computer network intrusions using TCP dump data for a local-area network (LAN) simulating a typical U.S. Air Force LAN. A connection is a sequence of TCP packets starting and ending at some well defined times, between which data flows to and from a source IP address to a target IP address under some well defined protocol. Each connection is labeled as either normal, or as an attack. 

There are 4 types of attacks in the dataset:
- DOS: denial-of-service, e.g. syn flood;
- R2L: unauthorized access from a remote machine, e.g. guessing password;
- U2R:  unauthorized access to local superuser (root) privileges;
- probing: surveillance and other probing, e.g., port scanning.
    
The dataset contains about 5 million connection records.

There are 3 types of features:
- basic features of individual connections, e.g. duration of connection
- content features within a connection, e.g. number of failed log in attempts
- traffic features within a 2 second window, e.g. number of connections to the same host as the current connection

The outlier detector is only using 40 out of 41 features.

## Train locally

Train on small dataset where you roughly know the fraction of outliers, defined by the "contamination" parameter.

In [ ]:
# define columns to keep
cols=['duration','protocol_type','flag','src_bytes','dst_bytes','land',
      'wrong_fragment','urgent','hot','num_failed_logins','logged_in',
      'num_compromised','root_shell','su_attempted','num_root','num_file_creations',
      'num_shells','num_access_files','num_outbound_cmds','is_host_login',
      'is_guest_login','count','srv_count','serror_rate','srv_serror_rate',
      'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate',
      'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate',
      'dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate',
      'dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
      'dst_host_srv_rerror_rate','target']
cols_str = str(cols)

In [ ]:
!python train.py \
--dataset 'kddcup99' \
--samples 50000 \
--keep_cols "$cols_str" \
--contamination .1 \
--n_estimators 100 \
--max_samples .8 \
--max_features 1. \
--save_path './models/'

## Test using Kubernetes cluster on GCP or Minikube

Run the outlier detector as a model or a transformer. If you want to run the anomaly detector as a transformer, change the SERVICE_TYPE variable from MODEL to TRANSFORMER [here](./.s2i/environment), set MODEL = False and change ```OutlierIsolationForest.py``` to:

```python
from CoreIsolationForest import CoreIsolationForest

class OutlierIsolationForest(CoreIsolationForest):
    """ Outlier detection using Isolation Forests.

    Parameters
    ----------
        threshold (float) : anomaly score threshold; scores below threshold are outliers
    """
    def __init__(self,threshold=0.,load_path='./models/'):

        super().__init__(threshold=threshold, load_path=load_path)
```

In [ ]:
MODEL = True

Pick Kubernetes cluster on GCP or Minikube.

In [ ]:
MINIKUBE = True

In [ ]:
if MINIKUBE:
    !minikube start --memory 4096
else:
    !gcloud container clusters get-credentials standard-cluster-1 --zone europe-west1-b --project seldon-demos

Create a cluster-wide cluster-admin role assigned to a service account named “default” in the namespace “kube-system”.

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin \
--serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

Add current context details to the configuration file in the seldon namespace.

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Create tiller service account and give it a cluster-wide cluster-admin role.

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

Check deployment rollout status and deploy seldon/spartakus helm charts.

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-operator --name seldon-core --set usage_metrics.enabled=true --namespace seldon-system

Check deployment rollout status for seldon core.

In [ ]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

Install Ambassador API gateway

In [ ]:
!helm install stable/ambassador --name ambassador --set image.tag=0.40.2

In [ ]:
!kubectl rollout status deployment.apps/ambassador

If Minikube used: create docker image for outlier detector inside Minikube using s2i. Besides the transformer image and the demo specific model image, the general model image for the Isolation Forest outlier detector is also available from Docker Hub as ***seldonio/outlier-if-model:0.1***.

In [ ]:
if MINIKUBE & MODEL:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-if-model-demo:0.1
elif MINIKUBE:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-if-transformer:0.1

Install outlier detector helm charts either as a model or transformer and set *threshold* hyperparameter value.

In [ ]:
if MODEL:
    !helm install ../../../helm-charts/seldon-od-model \
        --name outlier-detector \
        --namespace=seldon \
        --set model.type=isolationforest \
        --set model.isolationforest.image.name=seldonio/outlier-if-model-demo:0.1 \
        --set model.isolationforest.threshold=0 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set replicas=1
else:
    !helm install ../../../helm-charts/seldon-od-transformer \
        --name outlier-detector \
        --namespace=seldon \
        --set outlierDetection.enabled=true \
        --set outlierDetection.name=outlier-if \
        --set outlierDetection.type=isolationforest \
        --set outlierDetection.isolationforest.image.name=seldonio/outlier-if-transformer:0.1 \
        --set outlierDetection.isolationforest.threshold=0 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set model.image.name=seldonio/mock_classifier:1.0

## Port forward Ambassador

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Import rest requests, load data and test requests

In [ ]:
from utils import get_payload, rest_request_ambassador, send_feedback_rest, get_kdd_data, generate_batch

data = get_kdd_data(keep_cols=cols,percent10=True) # load dataset
print(data.shape)

Generate a random batch from the data

In [ ]:
import numpy as np

samples = 1
fraction_outlier = 0.
X, labels = generate_batch(data,samples,fraction_outlier)
print(X.shape)
print(labels.shape)

Test the rest requests with the generated data. It is important that the order of requests is respected. First we make predictions, then we get the "true" labels back using the feedback request. If we do not respect the order and eg keep making predictions without getting the feedback for each prediction, there will be a mismatch between the predicted and "true" labels. This will result in errors in the produced metrics.

In [ ]:
request = get_payload(X)

In [ ]:
response = rest_request_ambassador("outlier-detector","seldon",request,endpoint="localhost:8003")

If the outlier detector is used as a transformer, the output of the anomaly detection is added as part of the metadata. If it is used as a model, we send model feedback to retrieve custom performance metrics.

In [ ]:
if MODEL:
    send_feedback_rest("outlier-detector","seldon",request,response,0,labels,endpoint="localhost:8003")

## Analytics

Install the helm charts for prometheus and the grafana dashboard

In [ ]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

## Port forward Grafana dashboard

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:

Username : admin

password : password (as set when starting seldon-core-analytics above)

Import the outlier-detector-if dashboard from ../../../helm-charts/seldon-core-analytics/files/grafana/configs.

## Run simulation

- Sample random network intrusion data with a certain outlier probability.
- Get payload for the observation.
- Make a prediction.
- Send the "true" label with the feedback if the detector is run as a model.

It is important that the prediction-feedback order is maintained. Otherwise there will be a mismatch between the predicted and "true" labels.

View the progress on the grafana "Outlier Detection" dashboard. Most metrics need the outlier detector to be run as a model since they need model feedback.

In [ ]:
import time
n_requests = 100
samples = 1
for i in range(n_requests):
    fraction_outlier = .1
    X, labels = generate_batch(data,samples,fraction_outlier)
    request = get_payload(X)
    response = rest_request_ambassador("outlier-detector","seldon",request,endpoint="localhost:8003")
    if MODEL:
        send_feedback_rest("outlier-detector","seldon",request,response,0,labels,endpoint="localhost:8003")
    time.sleep(1)

In [ ]:
if MINIKUBE:
    !minikube delete